https://keras.io/guides/keras_nlp/getting_started/

In [8]:
!pip install -q --upgrade keras-nlp
!pip install -q --upgrade keras  # Upgrade to Keras 3.

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-decision-forests 1.8.1 requires wurlitzer, which is not installed.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-decision-forests 1.8.1 requires wurlitzer, which is not installed.
tensorflow 2.15.0 requires keras<2.16,>=2.15.0, but you have keras 3.3.3 which is incompatible.


In [10]:
import os

os.environ["KERAS_BACKEND"] = "jax"  # or "tensorflow" or "torch"

import keras_nlp
import keras

# Use mixed precision to speed up all training in this guide.
keras.mixed_precision.set_global_policy("mixed_float16")

In [11]:
!curl -O https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!tar -xf aclImdb_v1.tar.gz
!# Remove unsupervised examples
!rm -r aclImdb/train/unsup

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 80.2M  100 80.2M    0     0  4830k      0  0:00:17  0:00:17 --:--:-- 10.8M


In [12]:
BATCH_SIZE = 16
imdb_train = keras.utils.text_dataset_from_directory(
    "aclImdb/train",
    batch_size=BATCH_SIZE,
)
imdb_test = keras.utils.text_dataset_from_directory(
    "aclImdb/test",
    batch_size=BATCH_SIZE,
)

# Inspect first review
# Format is (review text tensor, label tensor)
print(imdb_train.unbatch().take(1).get_single_element())

Found 25000 files belonging to 2 classes.
Found 25000 files belonging to 2 classes.
(<tf.Tensor: shape=(), dtype=string, numpy=b"After reading several good reviews as well as hearing nice things about it by word of mouth I decided to rent Come Undone. I must say I was rather disappointed. The story was hard to follow because the film is set as a series of flashbacks between the present and recent past that are very poorly executed. The characters, despite the actors best efforts are flat and uninteresting. The sex is and nudity are more explicit than they need to be. I've never seen a film where they seemed so unnecessary to the plot. The ending is very anti-climatic and leaves many unanswered questions to a story line that wasn't explained well to begin with. In my opinion, a waste of time.">, <tf.Tensor: shape=(), dtype=int32, numpy=0>)


In [13]:
classifier = keras_nlp.models.BertClassifier.from_preset("bert_tiny_en_uncased_sst2")
# Note: batched inputs expected so must wrap string in iterable
classifier.predict(["I love modular workflows in keras-nlp!"])

Attaching 'metadata.json' from model 'keras/bert/keras/bert_tiny_en_uncased_sst2/4' to your Kaggle notebook...
Attaching 'metadata.json' from model 'keras/bert/keras/bert_tiny_en_uncased_sst2/4' to your Kaggle notebook...
Attaching 'task.json' from model 'keras/bert/keras/bert_tiny_en_uncased_sst2/4' to your Kaggle notebook...
Attaching 'task.json' from model 'keras/bert/keras/bert_tiny_en_uncased_sst2/4' to your Kaggle notebook...
Attaching 'task.json' from model 'keras/bert/keras/bert_tiny_en_uncased_sst2/4' to your Kaggle notebook...
Attaching 'task.json' from model 'keras/bert/keras/bert_tiny_en_uncased_sst2/4' to your Kaggle notebook...
Attaching 'task.weights.h5' from model 'keras/bert/keras/bert_tiny_en_uncased_sst2/4' to your Kaggle notebook...
Attaching 'task.weights.h5' from model 'keras/bert/keras/bert_tiny_en_uncased_sst2/4' to your Kaggle notebook...
/opt/conda/lib/python3.10/site-packages/keras/src/saving/saving_lib.py:415: UserWarning: Skipping variable loading for optim

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step


array([[-1.287,  1.393]], dtype=float16)

In [14]:
classifier.evaluate(imdb_test)


1563/1563 ━━━━━━━━━━━━━━━━━━━━ 700s 447ms/step - loss: 0.4592 - sparse_categorical_accuracy: 0.7888


[0.4550009071826935, 0.7903599739074707]

In [ ]:
classifier = keras_nlp.models.BertClassifier.from_preset(
    "bert_tiny_en_uncased",
    num_classes=2,
)
classifier.fit(
    imdb_train,
    validation_data=imdb_test,
    epochs=1,
)

Attaching 'metadata.json' from model 'keras/bert/keras/bert_tiny_en_uncased/2' to your Kaggle notebook...
Attaching 'metadata.json' from model 'keras/bert/keras/bert_tiny_en_uncased/2' to your Kaggle notebook...
Attaching 'task.json' from model 'keras/bert/keras/bert_tiny_en_uncased/2' to your Kaggle notebook...
Attaching 'config.json' from model 'keras/bert/keras/bert_tiny_en_uncased/2' to your Kaggle notebook...
Attaching 'metadata.json' from model 'keras/bert/keras/bert_tiny_en_uncased/2' to your Kaggle notebook...
Attaching 'metadata.json' from model 'keras/bert/keras/bert_tiny_en_uncased/2' to your Kaggle notebook...
Attaching 'config.json' from model 'keras/bert/keras/bert_tiny_en_uncased/2' to your Kaggle notebook...
Attaching 'config.json' from model 'keras/bert/keras/bert_tiny_en_uncased/2' to your Kaggle notebook...
Attaching 'model.weights.h5' from model 'keras/bert/keras/bert_tiny_en_uncased/2' to your Kaggle notebook...
Attaching 'metadata.json' from model 'keras/bert/kera

1328/1563 ━━━━━━━━━━━━━━━━━━━━ 5:33 1s/step - loss: 0.5302 - sparse_categorical_accuracy: 0.7189

In [ ]:
import tensorflow as tf

preprocessor = keras_nlp.models.BertPreprocessor.from_preset(
    "bert_tiny_en_uncased",
    sequence_length=512,
)

# Apply the preprocessor to every sample of train and test data using `map()`.
# [`tf.data.AUTOTUNE`](https://www.tensorflow.org/api_docs/python/tf/data/AUTOTUNE) and `prefetch()` are options to tune performance, see
# https://www.tensorflow.org/guide/data_performance for details.

# Note: only call `cache()` if you training data fits in CPU memory!
imdb_train_cached = (
    imdb_train.map(preprocessor, tf.data.AUTOTUNE).cache().prefetch(tf.data.AUTOTUNE)
)
imdb_test_cached = (
    imdb_test.map(preprocessor, tf.data.AUTOTUNE).cache().prefetch(tf.data.AUTOTUNE)
)

classifier = keras_nlp.models.BertClassifier.from_preset(
    "bert_tiny_en_uncased", preprocessor=None, num_classes=2
)
classifier.fit(
    imdb_train_cached,
    validation_data=imdb_test_cached,
    epochs=3,
)

In [ ]:
tokenizer = keras_nlp.models.BertTokenizer.from_preset("bert_tiny_en_uncased")
tokenizer(["I love modular workflows!", "Libraries over frameworks!"])

# Write your own packer or use one of our `Layers`
packer = keras_nlp.layers.MultiSegmentPacker(
    start_value=tokenizer.cls_token_id,
    end_value=tokenizer.sep_token_id,
    # Note: This cannot be longer than the preset's `sequence_length`, and there
    # is no check for a custom preprocessor!
    sequence_length=64,
)


# This function that takes a text sample `x` and its
# corresponding label `y` as input and converts the
# text into a format suitable for input into a BERT model.
def preprocessor(x, y):
    token_ids, segment_ids = packer(tokenizer(x))
    x = {
        "token_ids": token_ids,
        "segment_ids": segment_ids,
        "padding_mask": token_ids != 0,
    }
    return x, y


imdb_train_preprocessed = imdb_train.map(preprocessor, tf.data.AUTOTUNE).prefetch(
    tf.data.AUTOTUNE
)
imdb_test_preprocessed = imdb_test.map(preprocessor, tf.data.AUTOTUNE).prefetch(
    tf.data.AUTOTUNE
)

# Preprocessed example
print(imdb_train_preprocessed.unbatch().take(1).get_single_element())